In [1]:
%pylab inline
import pandas as pd
# csv files for answer responses
gdrive_path = "/Users/dorislee/Google Drive/Turn/user_study/evaluation_study/transcription_and_analysis/"

Populating the interactive namespace from numpy and matplotlib


In [2]:
user_task =  pd.read_csv(gdrive_path+"UserTaskAssignment.csv")

In [3]:
%load_ext autoreload
%autoreload 2
from paper_visualization import *

# Interested Retrieval

user_click_data.csv generated by running Parsing User Study Clickstream.ipynb

In [4]:
retreival = pd.read_csv(gdrive_path+"user_click_data.csv")

In [5]:
from tabulate import tabulate
import tabulate as T
del(T.LATEX_ESCAPE_RULES[u'\\'])

Equal weighting for interested + uninterested, 0 for no selection

In [6]:
def getDataset(x):
    if 'A' in x:
        return "Police"
    else:
        return "Autism"

In [8]:
retreival["Condition"] = retreival["task"].apply(lambda x:{1:"Storyboard",2:"Cluster",3:"BFS"}[int(x[1])])

In [11]:
retreival.groupby(["Condition","interested"]).count()

user  visualization  task
Condition  interested                               
BFS        interested        61             61    61
           not interested    20             20    20
           unselected        39             39    39
Cluster    interested        51             51    51
           not interested    22             22    22
           unselected        47             47    47
Storyboard interested        66             66    66
           not interested    10             10    10
           unselected        44             44    44

In [38]:
#averaging by user seems a little weird
# count_interested = retreival[retreival["interested"]=="interested"].groupby(["user","task"]).count().reset_index()[["user","visualization","task"]].rename(columns={"visualization":"interested_count"})

# count_selected = retreival[retreival["interested"]!="not interested"].groupby(["user","task"]).count().reset_index()[["user","visualization","task"]].rename(columns={"visualization":"total_selected_count"})

# df = count_interested.merge(count_selected)

# df["normalized_interested_rate"]=df["interested_count"]/df["total_selected_count"]

# df["Condition"] = df["task"].apply(lambda x:{1:"Storyboard",2:"Cluster",3:"BFS"}[int(x[1])])

# df.groupby("task").mean()

# df.groupby("Condition").mean()

In [75]:
data = [["Storyboard",66,10],['BFS',61,20],["Cluster",51,22]]

In [76]:
df_result = pd.DataFrame(data,columns=["Condition","Interested","Not Interested"])

In [77]:
df_result["Normalized Interested"] = df_result["Interested"]/(df_result["Interested"]+df_result["Not Interested"])

In [78]:
print (tabulate(df_result.transpose(),floatfmt=".2g", tablefmt="latex",headers="firstrow"))

\begin{tabular}{lrrr}
\hline
 Condition             &   Storyboard &   BFS &   Cluster \\
\hline
 Interested            &        66    & 61    &      51   \\
 Not Interested        &        10    & 20    &      22   \\
 Normalized Interested &         0.87 &  0.75 &       0.7 \\
\hline
\end{tabular}


-----------